In [6]:
2 + 2  # check if jupyter running)

4

In [7]:
import os
import random
from dataclasses import dataclass

import faker
import requests


@dataclass
class User:
    first_name: str
    last_name: str
    email: str
    password: str
    password2: str


@dataclass
class Post:
    id: int
    title: str


class RequestManager:
    def __init__(self, host: str = None, port: int = None):
        self.host = host or os.environ.get('HOST', 'http://app')
        self.port = port or os.environ.get('PORT', 8080)

    def auth(self, user: User) -> str:
        r = requests.post(
            f'{self.host}:{self.port}/api/users/token/',
            json={
                'email': user.email,
                'password': user.password,
            },
        )
        if r.status_code == 200:
            return r.json()['access']

    def post(self, url: str, data: dict = None, headers: dict = None) -> requests.Response:
        return requests.post(
            f'{self.host}:{self.port}/{url}',
            json=data,
            headers=headers,
        )

    def get(self, url: str, headers: dict = None) -> requests.Response:
        return requests.get(
            f'{self.host}:{self.port}/{url}',
            headers=headers,
        )


class BotManager:
    def __init__(self):
        self.number_of_users = int(os.environ.get('NUMBER_OF_USERS', 0))
        self.max_posts_per_user = int(os.environ.get('MAX_POSTS_PER_USER', 0))
        self.max_likes_per_user = int(os.environ.get('MAX_LIKES_PER_USER', 0))

        self.faker = faker.Faker()
        self.request_manager = RequestManager()

    def generate_users(self) -> list[User]:
        users = []
        for _ in range(self.number_of_users):
            password = self.faker.password()
            data = {
                'first_name': self.faker.first_name(),
                'last_name': self.faker.last_name(),
                'email': self.faker.email(),
                'password': password,
                'password2': password,
            }

            r = self.request_manager.post(
                url='api/users/sign-up/',
                data=data,
            )
            if r.status_code == 201:
                users.append(User(**data))
        return users

    def generate_posts(self, users: list):
        posts = []
        for user in users:
            user_post = []
            post_num = random.randint(0, self.max_posts_per_user)
            token = self.request_manager.auth(user)
            while len(user_post) != post_num:
                r = self.request_manager.post(
                    url='api/articles/',
                    data={'title': self.faker.sentence()[:10]},
                    headers={'Authorization': f'Bearer {token}'}
                )
                if r.status_code == 201:
                    user_post.append(Post(**r.json()))
            posts.extend(user_post)
        return posts

    def set_like_unlike(self, users: list[User], posts: list[Post]) -> None:
        actions = ('like', 'unlike')
        for user in users:
            likes_num = random.randint(0, self.max_likes_per_user)
            shuffled_posts = posts[:]
            random.shuffle(shuffled_posts)
            token = self.request_manager.auth(user)

            for _ in range(likes_num):
                try:
                    post = shuffled_posts.pop()
                except IndexError:
                    break

                action = random.choice(actions)
                self.request_manager.post(
                    url=f'api/articles/{post.id}/{action}/',
                    headers={'Authorization': f'Bearer {token}'}
                )

    def __call__(self):
        users = self.generate_users()
        posts = self.generate_posts(users)
        self.set_like_unlike(users, posts)


BotManager()()
print('Done!')

Done!
